### Task1b Regression with Feature Transformation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [2]:
df = pd.read_csv('./train.csv')
df.head(5)

,Id,y,x1,x2,x3,x4,x5
0,0,-7.912551,1.276266,-0.854628,1.623901,2.145311,2.037190
1,1,-5.728060,2.886639,0.888302,0.637899,1.148675,0.562217
2,2,-29.524046,3.171257,2.152310,-0.818812,0.861951,1.539840
3,3,-22.108731,-0.775282,2.314877,0.526552,-0.092230,0.749973
4,4,-15.234738,0.017706,2.031269,1.491334,0.553353,0.193640


In [3]:
y = df['y']
X = df._drop_axis(['Id','y'], axis=1)

#switch to numpy for the functions
y = np.array(y)
X = np.array(X)

#useful variables
num_rows, num_cols = X.shape 

In [4]:
#feature transormations
X_quad = np.square(X)
X_exp = np.exp(X)
X_cos = np.cos(X)
X_const = np.ones((num_rows, 20))

#concatenate all these transformations into transformed X
print(X.shape, X_quad.shape, X_exp.shape, X_cos.shape, X_const.shape)
X = np.concatenate((X, X_quad, X_exp, X_cos, X_const),axis=1)

#For some reason, it is impossible to append vector to numpy array, only matrix of same col
X_transformed = X[:,0:21]
print(X_transformed.shape)

(900, 5) (900, 5) (900, 5) (900, 5) (900, 20)
(900, 21)


In [9]:
#Set up regression
k = 9
reg = linear_model.LinearRegression()
min_reg = linear_model.LinearRegression()
weights = np.empty([k,21]) #9 folds so 9 models, 21 weights (w1...w21)
rmse = np.zeros(k)
i = 0

#Test if column of ones added
X_transformed[0,20]

1.0

### Now k-fold cross validation

In [10]:
#Split into k folds, each fold contains 900/k rows
kf = KFold(k)

for train_index, test_index in kf.split(X_transformed):
    X_train, X_test = X_transformed[train_index], X_transformed[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    reg.fit(X_train, y_train)
    y_pred = reg.predict(X_test)
    
    weights[i] = reg.coef_
    rmse[i] = mean_squared_error(y_pred, y_test)**0.5
    i = i + 1

print(rmse)

[10.76243819 12.19101628 10.1619707  10.60473813 10.28327809 10.51560906
 10.48735037]


#### Find model with lowest RMSE, send weights to output

In [11]:
index_of_min = np.argmin(rmse)
print(index_of_min)

2


In [12]:
np.savetxt("submit4.csv", weights[index_of_min], delimiter=",")